<a href="https://colab.research.google.com/github/krisibraynova01/HISB/blob/main/hisb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Прогнозиране на приоритет на инцидент въз основа на избрани атрибути**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('incident_event_log.csv')

In [ ]:
df.head()

**Подготвка на данните: Предварителна обработка на данни**

От визуализацията на данните се установява, че има някои колони, в които липсва информация, представена с „?“, или в почти всички записи, или в по-голямата част от записите. Трябва да се премахнат всички тези колони, където липсващата информация е в повече от една трета от общия брой редове



In [ ]:
max_bad = 50000 
df2 = df.loc[:,[(len(df[df[x]=='?']) < max_bad) for x in df.columns]]

In [ ]:
df2.info(verbose=False)

In [ ]:
df2 = df2.replace('?',np.nan)

In [ ]:
df2 = df2.dropna(axis = 0)

In [ ]:
df2.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75230 entries, 4 to 141711
Columns: 29 entries, number to closed_at
dtypes: bool(4), int64(3), object(22)
memory usage: 15.2+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df2.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by',
       'opened_at', 'sys_updated_by', 'sys_updated_at', 'contact_type',
       'location', 'category', 'subcategory', 'u_symptom', 'impact', 'urgency',
       'priority', 'assignment_group', 'assigned_to', 'knowledge',
       'u_priority_confirmation', 'notify', 'closed_code', 'resolved_by',
       'resolved_at', 'closed_at'],
      dtype='object')

Сега този набор от данни е много по-управляем по размер и няма липсваща информация.
Ще премахнем всички колони, които не влияят на резултата.
Колоните за въздействие и спешност заедно определят приоритета. Тъй като вече имаме приоритет, който ще бъде нашата променлива на резултата, можем да премахнем въздействието и спешността.

In [ ]:
df3= df2.drop(['number','notify', 'resolved_by','sys_updated_at','impact', 'urgency'], axis=1)

In [ ]:
print(df3['incident_state'].unique())

['New' 'Active' 'Awaiting User Info' 'Resolved' 'Closed'
 'Awaiting Problem' 'Awaiting Vendor' 'Awaiting Evidence']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Трябва да се запазят само редовете, които ни казват за състоянието на нов инцидент, тъй като се опитваме да предвидим приоритета на инцидентите, който се решава веднага след отварянето на инцидент (в повечето случаи)

In [ ]:
df_new = df3.drop(SNow3[SNow3['incident_state']!='New'].index)

In [ ]:
df_new.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15268 entries, 4 to 141390
Columns: 23 entries, incident_state to closed_at
dtypes: bool(4), int64(3), object(16)
memory usage: 2.4+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(df_new['priority'].unique())

['3 - Moderate' '4 - Low' '2 - High' '1 - Critical']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df_new.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['incident_state', 'active', 'reassignment_count', 'reopen_count',
       'sys_mod_count', 'made_sla', 'caller_id', 'opened_by', 'opened_at',
       'sys_updated_by', 'contact_type', 'location', 'category', 'subcategory',
       'u_symptom', 'priority', 'assignment_group', 'assigned_to', 'knowledge',
       'u_priority_confirmation', 'closed_code', 'resolved_at', 'closed_at'],
      dtype='object')

Изчисляване на времето за завършване на инцидентите в брой дни

In [ ]:
df_new['completion_days'] = pd.to_datetime(df_new['closed_at']) - pd.to_datetime(df_new['opened_at'])

In [ ]:
df_new.drop(['incident_state','active','resolved_at','closed_at','opened_at'], axis=1,inplace=True)

In [ ]:
df_new.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,sys_updated_by,contact_type,location,category,subcategory,u_symptom,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,closed_code,completion_days
4,0,0,0,True,Caller 2403,Opened by 397,Updated by 746,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,3 - Moderate,Group 70,Resolver 89,True,False,code 5,95 days 05:20:00
20,0,0,0,True,Caller 4491,Opened by 180,Updated by 340,Phone,Location 204,Category 9,Subcategory 97,Symptom 450,3 - Moderate,Group 25,Resolver 125,True,False,code 3,125 days 06:22:00
40,0,0,0,True,Caller 2838,Opened by 131,Updated by 265,Phone,Location 143,Category 53,Subcategory 168,Symptom 580,3 - Moderate,Group 70,Resolver 78,True,False,code 6,156 days 09:50:00
41,0,0,1,True,Caller 2838,Opened by 131,Updated by 265,Phone,Location 143,Category 53,Subcategory 168,Symptom 580,3 - Moderate,Group 70,Resolver 78,True,False,code 6,156 days 09:50:00
49,0,0,0,True,Caller 5323,Opened by 131,Updated by 265,Phone,Location 108,Category 44,Subcategory 229,Symptom 580,3 - Moderate,Group 5,Resolver 216,True,False,code 1,125 days 08:22:00



Сега трябва предвидим приоритета на инцидентите въз основа на някои характеристики. Изборът на функции се основава на личен опит в индустрията на ИТ услугите.

In [ ]:
df_new.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['reassignment_count', 'reopen_count', 'sys_mod_count', 'made_sla',
       'caller_id', 'opened_by', 'sys_updated_by', 'contact_type', 'location',
       'category', 'subcategory', 'u_symptom', 'priority', 'assignment_group',
       'assigned_to', 'knowledge', 'u_priority_confirmation', 'closed_code',
       'completion_days'],
      dtype='object')

In [ ]:
X = df_new[['caller_id', 'opened_by', 'sys_updated_by', 'contact_type', 'location', 'category', 'subcategory', 'u_symptom','u_priority_confirmation']]
y = df_new[['priority']]

**Създаване на модел**

За тестване на нашия модел ще използваме test_train_split, за да разделим данните в съотношение 70:30.

Резултатът от точността ще определи ефективността на модела

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,f1_score

In [ ]:
pd.options.mode.chained_assignment = None

**Числено кодиране на данните**

In [ ]:
#Numerical encoding
for col in ['caller_id', 'opened_by', 'sys_updated_by', 'location', 'category', 'subcategory','u_symptom']:
    X[col] = X[col].str.extract('(\\d+)').astype(np.int64)

# Label Encoding
enc= LabelEncoder()
for col in ['u_priority_confirmation', 'contact_type']:
    X.loc[:,col] = enc.fit_transform(X.loc[:,col])
y['priority'] = enc.fit_transform(y['priority'] ).astype(np.int64)

In [ ]:
X.head()

,caller_id,opened_by,sys_updated_by,contact_type,location,category,subcategory,u_symptom,u_priority_confirmation
4,2403,397,746,1,165,40,215,471,0
20,4491,180,340,1,204,9,97,450,0
40,2838,131,265,1,143,53,168,580,0
41,2838,131,265,1,143,53,168,580,0
49,5323,131,265,1,108,44,229,580,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=20)

In [ ]:
import warnings
warnings.filterwarnings('always') 

In [ ]:
print(f' The baseline of accuracy is {accuracy_score(y_test, np.full(y_test.shape, 3))}')
print(classification_report(y_test,np.full(y_test.shape, 3)))

 The baseline of accuracy is 0.030561012879284
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        64
           1       0.00      0.00      0.00       149
           2       0.00      0.00      0.00      4228
           3       0.03      1.00      0.06       140

    accuracy                           0.03      4581
   macro avg       0.01      0.25      0.01      4581
weighted avg       0.00      0.03      0.00      4581



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python

Базовата линия на точност е 0,030561012879284

In [ ]:
y_train.priority.value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2    9879
1     323
3     310
0     175
Name: priority, dtype: int64

Тъй като може да се види, че приоритет = 3 е непропорционално представен, имам нужда от някакъв механизъм за увеличаване на градиента.

Ще използваме класификатор за усилване на градиента, за да изградим или моделираме и ще базираме нашата прогноза на това.

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [ ]:
m_xgb = XGBClassifier()
m_xgb.fit(X_train, y_train)
y_predict_xgb = m_xgb.predict(X_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
y_predict_xgb = m_xgb.predict(X_test)

#The final report is:
f1 = f1_score(y_test,y_predict_xgb,average='macro')
print(f'The macro F1 score for initial XGB model:{f1}')
print(classification_report(y_test,y_predict_xgb))

The macro F1 score for initial XGB model:0.8136550631515472
              precision    recall  f1-score   support

           0       0.89      0.64      0.75        64
           1       0.88      0.60      0.72       149
           2       0.97      1.00      0.98      4228
           3       0.94      0.71      0.81       140

    accuracy                           0.97      4581
   macro avg       0.92      0.74      0.81      4581
weighted avg       0.97      0.97      0.97      4581



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Това изглежда вече е доста добър резултат!

In [ ]:
class_weights = list(class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train), y=y_train['priority']))


weights_array = np.ones(y_train.shape[0], dtype = 'float')
for i, val in enumerate(y_train['priority']):
    weights_array[i] = class_weights[val-1]

xgb_weighted_model = XGBClassifier()   
xgb_weighted_model.fit(X_train, y_train,sample_weight=weights_array)

In [ ]:
y_predict_weighted = xgb_weighted_model.predict(X_test) 
f1 = f1_score(y_test,y_predict_weighted,average='macro')

print(f'The F1 score for the Weighted model:{f1}')
print(classification_report(y_test,y_predict_weighted))